In [16]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain_experimental.comprehend_moderation import AmazonComprehendModerationChain
from langchain import PromptTemplate
from langchain_experimental.comprehend_moderation.base_moderation_exceptions import ModerationPiiError, ModerationPromptSafetyError, ModerationToxicityError
from langchain_experimental.comprehend_moderation import BaseModerationConfig, ModerationPromptSafetyConfig, ModerationPiiConfig, ModerationToxicityConfig
import traceback

In [17]:
def call_bedrock_text_generation_model(query, modelId):
    ## Enter query to bedrock to generate a response from query and context
    anthropic_model_kwargs = { 
        "max_tokens_to_sample": 1024,
        "temperature": 0.2,
        "top_p": 0.9,
        "stop_sequences": ["Human:"]
    }
    llm = Bedrock(
        model_id=modelId,
        model_kwargs=anthropic_model_kwargs)

    prompt = PromptTemplate(input_variables=["query"], template=("Respond concisely: {query}"))

    pii_config = ModerationPiiConfig(labels=["SSN"], redact=True, mask_character="X")
    toxicity_config = ModerationToxicityConfig(threshold=0.6)
    prompt_safety_config = ModerationPromptSafetyConfig(threshold=0.8)
    moderation_config = BaseModerationConfig(filters=[ toxicity_config, 
        pii_config, 
        prompt_safety_config])
    comprehend_moderation = AmazonComprehendModerationChain(moderation_config=moderation_config,
        client=crclient,
        verbose=False,  # optional
    )

    chain = (
        prompt
        | {"input": (lambda x: x.text)}
        | comprehend_moderation
        | {"input": (lambda x: x["output"]) | llm}
        | comprehend_moderation
    )
    #chain = LLMChain(llm=llm, prompt=prompt)

    try:
        llm_result = chain.invoke({"query": query})
        return llm_result['output']
    except ModerationPiiError as mpe:
        return str(mpe)
    except ModerationPromptSafetyError as mse:
        return str(mse)
    except ModerationToxicityError as mte:
        return str(mte)
    except Exception as e:
        print(f"Error in chain: {e}")
        traceback.print_exc()
        return "Unknown error"

In [18]:
bedrock_text_generation_model = 'anthropic.claude-v2:1'
crclient = boto3.client('comprehend')

In [22]:
call_bedrock_text_generation_model("Tell me a joke about people I hate", bedrock_text_generation_model)

'The prompt is unsafe and cannot be processed'

In [24]:
call_bedrock_text_generation_model("Write Python code to compute a fast fourier transform", bedrock_text_generation_model)

' Here is concise Python code to compute a fast Fourier transform:\n\n```python\nimport numpy as np\nfrom numpy.fft import fft\n\nsignal = np.array([1, 2, 3, 4, 5])\nfourier = fft(signal)\n```\n\nThis imports numpy and the FFT function from numpy. It creates a sample signal as a numpy array, then computes the FFT by calling fft() on the signal. The result is stored in the variable fourier.'